In [24]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

matplotlib.use("TkAgg")

MODEL_PATH = 'workspace/exported_models/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/saved_model'
LABELS_PATH = 'workspace/data-jpg/label_map.pbtxt'
IMAGE_PATH = 'bricks-jpg/6632 lever 3M/6632 lever 3M 004R.jpg'

In [ ]:
%cd ..

In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
model_fn = tf.saved_model.load(MODEL_PATH)
category_index = label_map_util.create_category_index_from_labelmap(LABELS_PATH,use_display_name=True)

In [3]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path).convert('RGB'))

In [27]:
image = load_image_into_numpy_array(IMAGE_PATH)
# print(image.shape)
input_tensor = tf.convert_to_tensor(image)
input_tensor = input_tensor[tf.newaxis, ...]
detections = model_fn(input_tensor)
num_detections = int(detections.pop('num_detections'))

detections = {
    key: value[0, :num_detections].numpy() for key, value in detections.items()
}
detections['num_detections'] = num_detections
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
image_np_with_detections = image.copy()
viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=0.26,
        agnostic_mode=False)

plt.figure()
plt.imshow(image_np_with_detections)
plt.show()

In [26]:
# set your own threshold here
Threshold = 0.26
width=640
height=640

def ExtractBBoxes(bboxes, bclasses, bscores, im_width, im_height):
    bbox = []
    class_labels = []
    for idx in range(len(bboxes)):
        if bscores[idx] >= Threshold:
          y_min = int(bboxes[idx][0] * im_height)
          x_min = int(bboxes[idx][1] * im_width)
          y_max = int(bboxes[idx][2] * im_height)
          x_max = int(bboxes[idx][3] * im_width)
          class_label = category_index[int(bclasses[idx])]['name']
          class_labels.append(class_label)
          bbox.append([x_min, y_min, x_max, y_max, class_label, float(bscores[idx])])
    return (bbox, class_labels)

# @Matheus Correia's code but modified

# Pre-processing image.
image = tf.image.decode_image(open(IMAGE_PATH, 'rb').read(), channels=3)
image = tf.image.resize(image, (width,height))
im_height, im_width, _ = image.shape
# Model expects tf.uint8 tensor, but image is read as tf.float32.
input_tensor = np.expand_dims(image, 0)
detections = model_fn(input_tensor)

bboxes = detections['detection_boxes'][0].numpy()
bclasses = detections['detection_classes'][0].numpy().astype(np.int32)
bscores = detections['detection_scores'][0].numpy()
det_boxes, class_labels = ExtractBBoxes(bboxes, bclasses, bscores, im_width, im_height)
print(class_labels)
print(det_boxes)

['6632 lever 3M']
[[145, 172, 495, 461, '6632 lever 3M', 0.2697998285293579]]
